In [ ]:
from utils import *
from yggdrasil.runner import run
from yggdrasil import tools

# Toy Plant Model

For the exercises today, we will be using a toy functional-structural plant model that makes the above plant mesh grow over time.

In the first version of this model, plant_v0.py (shown below), the growth is very simple and scales directly with time.

In [ ]:
from models import plant_v0
tools.display_source(plant_v0.run, number_lines=True)

The cell below runs this model directly (it is written in Python) and displays the resulting mesh at the end of the run. As you can see the mesh has 'grown' vertically.

In [ ]:
mesh = plant_v0.run(mesh, 0, 28, 1)
mesh.show()

However, we can also run the model using `yggdrasil` if we have a yaml configuration file for it. The cell below displays the configuration file for this model.

In [ ]:
tools.display_source('yamls/plant_v0.yml', number_lines=True)

The following code allows yggdrasil to run the model using the Python API. This is usually done via the `yggrun` command line utility (e.g. `yggrun yamls/plant_v0.yml`). Because this method of running the model requires loading the `yggdrasil` library, it takes a bit longer to run, but would not be significantly larger for a model with a more realistic (and computationally intensive) calculation.

In [ ]:
run('yamls/plant_v0.yml')
display_last_timestep()

But now we want to add information about other processes to our plant model...

# Call to Light Model

We can add information by calling other models as functions. For example, consider the toy light model below. It is a function that takes height and time as input and returns the light. 

Height can be a scalar or an array, but it is computationally more efficient to pass the 

In [ ]:
from models import light
tools.display_source(light.light, number_lines=True)

Because the light model is a function, yggdrasil can automatically wrap it with interface send and receive calls. Therefore, the YAML specification file for this model is relativly simple and the light model itself does not need to be modified at all.

In [ ]:
tools.display_source('yamls/light.yml', number_lines=True)

The YAML specification file for this version of the plant model differs in that it specifies that it is a client of the model with the name "light" and that the light output should be directed to a file if it is not connected to a model.

In [ ]:
tools.display_source_diff('yamls/plant_v0.yml', 'yamls/plant_v1.yml', number_lines=True)

Then we can add a interface call to the plant model "client" that calls the light model "server". In addition to the client comm, I have also added an output comm for the light values so that we can plot the light values.

In [ ]:
from models import plant_v1
tools.display_source_diff(plant_v0.run, plant_v1.run, number_lines=True)

We can run the integration of these two models by providing the paths to the YAML specification models to the yggdrasil API. The command to run this integration on the command line would be `yggrun yamls/plant_v1.yml yamls/light.yml`.

In [ ]:
run(['yamls/plant_v1.yml', 'yamls/light.yml'], production_run=True)

The cell below displays the resulting mesh with color mapped to the light. We can see that the growth tracked by this version of the model is much more subtle.

In [ ]:
display_last_timestep(with_light=True)

## The light model is easily replaced with other models

Feel free to try running one or more of them. They have been coded to produce the same results as the Python version of the light model.

### C++ Model

In [ ]:
tools.display_source('yamls/light_cpp.yml', number_lines=True)
tools.display_source('models/light.cpp', number_lines=True)
run(['yamls/plant_v1.yml', 'yamls/light_cpp.yml'], production_run=True)
display_last_timestep(with_light=True)

### R Model

In [ ]:
tools.display_source('yamls/light_R.yml', number_lines=True)
tools.display_source('models/light.R', number_lines=True)
run(['yamls/plant_v1.yml', 'yamls/light_R.yml'], production_run=True)
display_last_timestep(with_light=True)

### Fortran Model

In [ ]:
tools.display_source('yamls/light_fortran.yml', number_lines=True)
tools.display_source('models/light.f90', number_lines=True)
run(['yamls/plant_v1.yml', 'yamls/light_fortran.yml'], production_run=True)
display_last_timestep(with_light=True)